In [2]:
import os
import nibabel as nib
import trimesh
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import API as allen

from statistics import mean


In [3]:
def Obj_Gifti(loadFolder: str = 'Raw',
                saveFolder: str = 'Gii'
              ):
    # converts raw meshes from allen api from .obj to .gii types
    # targets main/Region_Mesh/Raw 

    meshFolder = 'Region_Mesh'  # all meshes
    rawMesh = os.listdir(f'{meshFolder}/{loadFolder}/')  # list of raw mesh files
    giiFolder = f'./{meshFolder}/{saveFolder}'     # save location


    # make Gii subfolder if doesn't exist
    meshTypes = os.listdir(meshFolder)
    if 'Gii' not in meshTypes:
        os.mkdir(f'{meshFolder}/Gii')


    for meshFile in rawMesh:
        mesh = trimesh.load(f'./{meshFolder}/Raw/{meshFile}')

        # split out vertices
        giiVertices = nib.gifti.GiftiDataArray(data=mesh.vertices.astype(np.float32),
                                               intent='NIFTI_INTENT_POINTSET')
        
        # split out triangle faces
        # print(mesh.faces.astype(np.float32))
        giiFaces = nib.gifti.GiftiDataArray(data=mesh.faces.astype(np.float32),
                                            intent='NIFTI_INTENT_POINTSET')

        # merge into single gifti image 
        giiImg = nib.gifti.GiftiImage(darrays=[giiVertices, giiFaces])
    
        # print(f'{giiFolder}/{meshFile[:-4]}.gii')

        nib.save(giiImg, f'{giiFolder}/{meshFile[:-4]}.gii')

In [4]:
def Open_Gifti(node: int = None,
               filePath: str = None
               ):
    
    # requires one of node or filePath
    if node is not None:
        file = nib.load(f'Region_Mesh/Gii/{node}.gii')
    
    if filePath is not None:
        file = nib.load(filePath)

    return file

In [5]:
def Open_Raw(node: int,
             transform = True,
             rotation: float = np.pi
             ):
    
    """
    node : id of region to open
    transform : transform the mesh so that the centroid of the cortex is at [0, 0, 0] and camera can be reset to the preferred view

    returns: trimesh object

    opens raw .obj file from Allen Institute, easier to work with in pymesh than a gifti
    """

    file = trimesh.load(f'./Region_Mesh/Raw/{node}.obj')

    if transform is True:
        # centroid of isocortex
        translation = -1 * np.asarray([5815.30949447, 501.96668849, 5692.50297221])
        # rotation origin and amount
        rotation = trimesh.transformations.rotation_matrix(1*rotation, [0, 1, 0], [0, 0, 0])
        # scale factor
        scale = np.eye(4)

        file.apply_translation(translation)
        # file.apply_transform(rotation)



    return file

In [6]:
# hey so uh turns out this already exists in mesh.slice_plane()
def __Hemisphere_Filter(nodeId: int|list[int],
                      axis: str = 'z',
                      plane: int = 0,
                      direction: int = 1):
    """
        nodeId: ids of node(s) to filter by hemisphere
        axis: which axis to filter along, best used when transform_zero is True
        plane: axis coordinate of plane to slice through, defaults to centroid with transform_zero True
        direction: which side of slice to keep, corresponds to negative/positive values
            x: positive posterior, negative anterior (coronal slice)
            y: positive ventral, negative dorsal (horizontal slize)
            z: positive/negative lateral, 0 medial (midsagittal slice)
        
        Opens raw .obj file(s) and filters vertices and faces by dimension given
    """

    # change given axis to fortran indexing
    axis_map = {'x': 0,
                'y': 1,
                'z': 2
                }
    axis = axis_map[axis]

    # filter single region
    if type(nodeId) == int:
        mesh = Open_Raw(node=nodeId)

        # mask for filtering faces
        if direction >= 0:
            face_mask = mesh.triangles_center[:, axis] >= plane
        elif direction < 0:
            face_mask = mesh.triangles_center[:, axis] <= plane

        # vertices are updated automatically
        mesh.update_faces(face_mask)


    # filter multiple regions
    if type(nodeId) == list:
        file = []
        for node in nodeId:
            file.append(Open_Gifti(node=nodeId))

    # print(points)

    return mesh

In [7]:
def Hemisphere_Filter(region: int = None,
                      mesh: trimesh.Trimesh = None,
                      hemisphere: str = 'left'):
    """
    Filters meshes to only single side of brain
    at leat one of id and mesh should be provided

    region: id of brain region, if provided loads from file
    mesh: Trimesh object of region
    hemisphere: which side of brain to keep, should be either 'left' or 'right
    """

    if region is not None:
        mesh = Open_Raw(region)
    elif mesh is None:
        print('Neither region id or mesh provided')
        return None
    

    mesh_bodies_temp = []
    mesh_bodies = mesh.split()
    for mesh in mesh_bodies:
        # left side has -ve coords
        if hemisphere == 'left' and mesh.center_mass[2] < 0:
            mesh_bodies_temp.append(mesh)

        if hemisphere == 'right' and mesh.center_mass[2] > 0:
            mesh_bodies_temp.append(mesh)

        if mesh.center_mass[2] == 0:
            print(f'mesh has centroid at z: 0, unable to decide hemisphere')
    
    mesh_new = trimesh.util.concatenate(mesh_bodies_temp)

    return mesh_new
    

In [11]:
# plot child regions of isocortex
tree = allen.Reference_Tree()
child_regions = tree.children([315])[0]
plotter = pv.Plotter()

print(child_regions[0])

for child in child_regions:
    # mesh = Hemisphere_Filter(child['id'])
    mesh = Hemisphere_Filter(child['id'], hemisphere='left')

    plotter.add_mesh(mesh, color=child['rgb_triplet'], label=child['name'])

plotter.add_legend(size=(0.5, 0.5), loc='lower left')
plotter.view_vector((0, 0, -1), viewup=(0, -1, 0))
plotter.show_grid()
plotter.show()



{'acronym': 'FRP', 'graph_id': 1, 'graph_order': 6, 'id': 184, 'name': 'Frontal pole, cerebral cortex', 'structure_id_path': [997, 8, 567, 688, 695, 315, 184], 'structure_set_ids': [3, 112905828, 688152357, 691663206, 687527945, 12, 184527634, 167587189, 112905813, 114512891], 'rgb_triplet': [38, 143, 69]}


Widget(value='<iframe src="http://localhost:52917/index.html?ui=P_0x19b8fbaead0_2&reconnect=auto" class="pyvis…

In [9]:
# plotting frontal pole of cerebral cortex
# region ids
FRP = [ 68, 667, 526157192, 526157196, 526322264]

# colour map
colors = plt.cm.Set1(np.linspace(0, 1, len(FRP)))
print(colors)
plotter = pv.Plotter()

for c, region in enumerate(FRP):
    mesh = Hemisphere_Filter(region, hemisphere='left')

    plotter.add_mesh(mesh, color=colors[c])

plotter.show()


[[0.89411765 0.10196078 0.10980392 1.        ]
 [0.30196078 0.68627451 0.29019608 1.        ]
 [1.         0.49803922 0.         1.        ]
 [0.65098039 0.3372549  0.15686275 1.        ]
 [0.6        0.6        0.6        1.        ]]


Widget(value='<iframe src="http://localhost:52917/index.html?ui=P_0x19b99cef210_1&reconnect=auto" class="pyvis…

In [10]:
mesh = Open_Raw(315)

mesh = mesh.slice_plane(np.array([0, 0, 0]), np.array([0, 0, 1]), cap=True)

mesh.show()